### Type Hinting
Please use type hints wherever possible. This will help improve the readability and maintainability of your code.

Incorrect or incomplete type hinting **will** result in penalties if trivial type hints are used incorrectly (for example, `list` is used instead of `float`, where the variable is obviously a number).




## Problem 1 [25 points]

Create an abstract base class `Expression` that represents a mathematical expression. This class should require methods for evaluating the expression (given a dictionary of variable values), for computing its symbolic derivative with respect to a variable. Also, it should require pretty printing and formatting for expressions. Then, implement concrete classes for constants, variables and binary operations (addition, subtraction, multiplication and division).

In [1]:
eval("(2 * x + 3) * (x - 5) / (1 - x)".replace('x','12'))

-17.181818181818183

In [29]:
from abc import ABC, abstractmethod


class Expression(ABC):

    @abstractmethod
    def evaluate(self, values_dict: dict):
        pass

    @abstractmethod
    def derivative(self, with_respect_to: str):
        pass


class Constant(Expression):
    def __init__(self, value):
        self.value = value

    def evaluate(self, values_dict={}):
        return self.value

    def __repr__(self):
        return f"Constant(value={self.value})"

    def __str__(self):
        return str(self.value)
    
    def derivative(self, respect_to: str):
        return Constant(0)


class Variable(Expression):
    def __init__(self, name):
        self.name = name

    def evaluate(self, x_str):
        return x_str[self.name]

    def derivative(self, respect_to: str):
        if respect_to == self.name:
            return Constant(1)
        else:
            return Constant(0)


    def __repr__(self):
        return f"Variable(name={self.name})"

    def __str__(self):
        return str(self.name)


class Add(Expression):
    def __init__(self, expr_1: Expression, expr_2: Expression):
        self.expr_1 = expr_1
        self.expr_2 = expr_2

    def evaluate(self, values_dict: dict):
        return self.expr_1.evaluate(values_dict) + self.expr_2.evaluate(values_dict)
    
    def derivative(self, respect_to: str):
        return Add(self.expr_1.derivative(respect_to), self.expr_2.derivative(respect_to))

    def __repr__(self):
        return f"Add(expr_1={self.expr_1.__str__()}, expr_2={self.expr_2.__str__()})"

    def __str__(self):
        return f"({self.expr_1.__str__()} + {self.expr_2.__str__()})"


class Subtract(Expression):
    def __init__(self, expr_1: Expression, expr_2: Expression):
        self.expr_1 = expr_1
        self.expr_2 = expr_2

    def evaluate(self, values_dict: dict):
        return self.expr_1.evaluate(values_dict) - self.expr_2.evaluate(values_dict)


    def derivative(self, respect_to: str):
        return Subtract(self.expr_1.derivative(respect_to), self.expr_2.derivative(respect_to))

    def __repr__(self):
        return (
            f"Subtract(expr_1={self.expr_1.__str__()}, expr_2={self.expr_2.__str__()})"
        )

    def __str__(self):
        return f"({self.expr_1.__str__()} - {self.expr_2.__str__()})"


class Multiply(Expression):
    def __init__(self, expr_1: Expression, expr_2: Expression):
        self.expr_1 = expr_1
        self.expr_2 = expr_2

    def evaluate(self, values_dict: dict):
        return self.expr_1.evaluate(values_dict) * self.expr_2.evaluate(values_dict)

    def derivative(self, respect_to: str):
        first_part = Multiply(self.expr_1.derivative(respect_to), self.expr_2)
        second_part = Multiply(self.expr_1, self.expr_2.derivative(respect_to))

        return Add(first_part, second_part)

    def __repr__(self):
        return (
            f"Multiply(expr_1={self.expr_1.__str__()}, expr_2={self.expr_2.__str__()})"
        )

    def __str__(self):
        return f"({self.expr_1.__str__()} * {self.expr_2.__str__()})"

class Divide(Expression):
    def __init__(self, expr_1: Expression, expr_2: Expression):
        self.expr_1 = expr_1
        self.expr_2 = expr_2

    def evaluate(self, values_dict: dict):
        return self.expr_1.evaluate(values_dict) / self.expr_2.evaluate(values_dict)
    
    def derivative(self, respect_to: str):
        first_part = Multiply(self.expr_1.derivative(respect_to), self.expr_2)
        second_part = Multiply(self.expr_1, self.expr_2.derivative(respect_to))


        return Divide(Subtract(first_part, second_part), Multiply(self.expr_2, self.expr_2))

    def __repr__(self):
        return (
            f"Divide(expr_1={self.expr_1.__str__()}, expr_2={self.expr_2.__str__()})"
        )

    def __str__(self):
        return f"({self.expr_1.__str__()} / {self.expr_2.__str__()})"


# # expr = Divide(
# #     Multiply(
# #         Add(Multiply(Constant(2), Variable("x")), Constant(3)),
# #         Subtract(Variable("x"), Constant(5))
# #     ),
# #     Subtract(Constant(1), Variable("x"))
# # )

# # print(expr) # (2 * x + 3) * (x - 5) / (1 - x)

# # assert expr.evaluate({"x": 3}) == 9.0

# # derivative_expr = expr.derivative("x")

# # assert derivative_expr.evaluate({"x": 3}) == -7.0

In [30]:
# add = Subtract(Constant(3), Variable("x"))
# add.evaluate({"x": 12})
expr = Divide(
    Multiply(
        Add(Multiply(Constant(2), Variable("x")), Constant(3)),
        Subtract(Variable("x"), Constant(5))
    ),
    Subtract(Constant(1), Variable("x"))
)

expr.evaluate({"x": 3}) == 9.0
print(expr)

derivative_expr = expr.derivative("x")

# assert derivative_expr.evaluate({"x": 3}) == -7.0

derivative_expr.evaluate({"x": 3})

((((2 * x) + 3) * (x - 5)) / (1 - x))


-7.0